In [25]:
from sympy import *                 # use symbolic algebra
init_printing(use_latex='mathjax')  # print nice math (use_latex arg is workaround for FormatterWarning messages)
from IPython import display

In [26]:
from sdutil2 import SD, FEF

In [27]:
# Slope deflection equations:
# theta_a,b,c,d - rotations of joints
# Delta_ab - horizontal displacement of joint b (and c) relative to a (and d)
var('EI theta_a theta_b theta_c theta_d Delta_ab')
Mab,Mba,Vab,Vba = SD(4,EI,theta_a,theta_b,0) + FEF.udl(4,20)
Mbc,Mcb,Vbc,Vcb = SD(6,2*EI,theta_b,theta_c, 0) + FEF.p(6,80,2) + FEF.p(6,80,4)
Mcd,Mdc,Vcd,Vdc = SD(4,EI,theta_c,theta_d, 0) + FEF.udl(4,15)

In [4]:
help(SD)

Help on function SD in module sdutil2:

SD(L, EI, t0, t1, d01=0)
    Return the end moments and shears of a segment as computed by the slope
    deflection equations.
       L = length,
       EI = stiffness,
       t0 = rotation at end 0 (left),
       t1 = rotation at end 1,
       d01 = relative lateral displacment end 0 to end 1.
    SD sign convention: moments and rotation are +ive CW,
    relative displacement d01 is +ive for CW rotation of chord.



In [5]:
help(FEF)

Help on class FEF in module sdutil2:

class FEF(builtins.object)
 |  Methods to calculate the fixed end forces, (moments and shears),
 |  due to loads of various types.  All methods here return an EF object,
 |  which contains the span length, the moments at either end, and the shears
 |  at either end.
 |
 |  Static methods defined here:
 |
 |  cm(L, M, a)
 |      Return the FEMs and shears due to a single concentrated moment of M
 |      a distance a from the left support.
 |
 |  lvl(L, w1, w2=None, a=None, b=None, c=None)
 |      Return the FEMs and shears due to a linearly varying distributed load over
 |      a portion of the span.  Usage:
 |
 |        FEF.lvl(L,w1,w2,a,b,c)
 |
 |      L = span, w1 = intensity at left, w2 = intensity at right, a = distance from
 |      left end to left edge of load, b = length of load, c = distance from right
 |      edge of load to right end of span.
 |
 |  p(L, P, a)
 |      Return FEMs and shears due to a single concentrated force, P, a distanc

In [28]:
Mab

EI⋅(4⋅θₐ + 2⋅θ_b)                   
───────────────── - 26.6666666666667
        4                           

In [29]:
eqns = [ Mba+Mbc, Mcb+Mcd, theta_d, theta_a, Delta_ab, Delta_bc, Delta_cd]
soln = solve(eqns,[theta_a,theta_b,theta_c,theta_d])
soln

⎧              48.8888888888889       -51.1111111111111           ⎫
⎨θₐ: 0.0, θ_b: ────────────────, θ_c: ──────────────────, θ_d: 0.0⎬
⎩                     EI                      EI                  ⎭

In [30]:
{v:eval(v).subs(soln).n(4) for v in 'Mab,Mba,Vab,Vba,Vbc,Vcb,Vcd,Vdc,Mbc,Mcb,Mcd,Mdc'.split(',')}

{'Mab': -2.222,
 'Mba': 75.56,
 'Vab': 21.67,
 'Vba': -58.33,
 'Vbc': 80.74,
 'Vcb': -79.26,
 'Vcd': 49.17,
 'Vdc': -10.83,
 'Mbc': -75.56,
 'Mcb': 71.11,
 'Mcd': -71.11,
 'Mdc': -5.556}

In [31]:
Ra= Vab 
Rb= -Vba + Vbc
Rc= -Vcb + Vcd
Rd= -Vdc
Md = Mdc
Ma = Mab
[v.subs(soln).n(4) for v in [Ra, Rb, Rc, Rd, Md, Ma]]

[21.67, 139.1, 128.4, 10.83, -5.556, -2.222]

In [32]:
# sum Vertical forces
(Ra + Rb + Rc + Rd -300).subs(soln)

0

In [33]:
# sum of moments about A
((Ma)+(20*4*2)+(-Rb*4)+(80*6)+(80*8)+(-Rc*10)+(15*4*12)+(-Rd*14)+(Md)).subs(soln).n(4)

5.684e-14